### BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding   

#### 摘要

 我们介绍一个新的语言表述模型称为BERT，来自 Transformer 的双向编码器表征。不像最近的语言表述模型 ，BERT 旨在基于所有层的左、右语境来预训练深度双向表征。结果，预训练BERT表述是用一个额外的输出层能够被优调来创造广泛的最优模型对于广泛任务，例如问答和语言推理，没有本质特定任务结构修改。

BERT概念上是简单和时间上是有效的。它在11个自然语言处理任务上获得最先进的结果：GLUE基准提升到80.4%（7.6%绝对提升），MultiNLI准确率提升到86.7%(5.6绝对提升)，SQuAD v1.1问答测试的F1提升为93.2（1.5绝对提升),超过人类能力2.0。

#### 介绍

语言模型预训练显示对于提升自然语言处理任务是有效的。这些任务包括句子级别的任务例如自然语言推理和释义，目的是为了预测句子间关系通过整体分析它们，也有些是词级别任务例如命名实体识别，SQuAD问答。

有两个已存在策略用于预训练语言表述为下流任务：基于特征和微调。基于特征方法例如ELMo：使用特定任务结构包括预训练表述作为额外的特征。优调方法例如生成预训练转换，介绍最小的特定任务参数，训练在下流任务通过简单优调预训练参数。在之前工作，这两种分享相同目标函数在预训练阶段，它们使用单向语言模型来学习一般语言描述。

我们讨论当前技术严重限制预训练表述的能力，特别对于优调方法。主要限制是标准语言模型是单向的，这限制了架构的选择被使用在预训练阶段。举个例子，在OpenAI GPT，作者们使用从左到右架构，每个词只能注意之前词在Transformer的自注意力层。这些限制对于句子级别任务是欠优化的，可能是毁灭当应用基于优调的词级别任务，例如SQuAD问答，它是重要的两个方向整合上下文。

在这篇文章，我们优化基于优调的方法通过提出BERT：来自Transformers的双向编码描述。BERT处理之前提到的单方向约束通过提出一个新预训练目标：掩饰语言模型MLM，受完形填空任务所激励。掩饰语言模型随机掩饰输入的一些词，目标是基于它的上下文来预测原始词典id。不像从左到右语言模型预训练，MLM目标允许表述混合左右上下文，允许我们预训练一个深度双向Transformer。此外，掩饰语言模型，我们介绍下一个句子任务合并预训练文本对描述。

我们文章的贡献如下：

+ 我们阐述双向预训练对于语言表述的重要性。不像Radford et al.，它使用单向语言模型进行预训练，BERT使用掩饰语言模型来使能预训练深度双向表述。也与Peters et al.相反，它使用浅连接独立训练的从左到右和从右到左语言模型。
+ 我们显示预训练表述不需要很多特定任务架构的繁重的特征工程。BERT是第一个基于优调表述模型获得最先进性能在大量句子级别和词级别任务，超过很多带有特定任务架构的系统。
+ BERT在11个任务中获得最先进的性能。

#### 相关工作

##### 基于特征的方法

学习广泛应用词描述成为一个活跃的研究区域。预训练词向量被认为一个现代NLP系统完整的部分，提供显著提升词向量学习。这些方法被一般化为更高粒度，例如句子嵌入或者段落嵌入。正如传统词向量，这些学习表示也被典型用于下流模型的特征。ELMo生成传统词向量使用不同维度。他们提出抽取上下文敏感的特征。当整合上下文词嵌入使用已存在特定任务架构，ELMo在几个主要基准获得最先进的性能。

#### BERT

##### 模型架构

BERT 的模型架构是一个多层双向 Transformer 编码器，基于 Vaswani 等人 (2017) 描述的原始实现，在 tensor2tensor 库中发布。由于 Transformer 的使用最近变得很普遍，而且我们的实现与原始版本实际相同，我们将不再赘述模型架构的背景。

在本文中，我们将层数（即 Transformer 块）表示为 L，将隐藏尺寸表示为 H、自注意力头数表示为 A。在所有实验中，我们将前馈/滤波器尺寸设置为 4H，即 H=768 时为 3072，H=1024 时为 4096。我们主要报告在两种模型尺寸上的结果：

- BERTBASE: L=12, H=768, A=12, 总参数=110M 
- BERTLARGE: L=24, H=1024, A=16, 总参数=340M

为了比较，BERTBASE 的模型尺寸与 OpenAI GPT 相当。然而，BERT Transformer 使用双向自注意力机制，而 GPT Transformer 使用受限的自注意力机制，导致每个 token 只能关注其左侧的语境。我们注意到，双向 Transformer 在文献中通常称为「Transformer 编码器」，而只关注左侧语境的版本则因能用于文本生成而被称为「Transformer 解码器」。图 1 直观显示了 BERT、OpenAI GPT 和 ELMo 的比较结果。

![1539486494140](F:\远传\撰写论文\BERT1.png)

**预训练任务**

与 Peters 等人 (2018) 和 Radford 等人 (2018) 不同，我们不使用传统的从左到右或从右到左的语言模型来预训练 BERT，而是使用两个新型无监督预测任务。

任务 #1：Masked LM

为了训练深度双向表征，我们采取了一个直接的方法，随机遮蔽输入 token 的某些部分，然后预测被遮住的 token。我们将这一步骤称为「masked LM」(MLM)，不过它在文献中通常被称为 Cloze 任务 (Taylor, 1953)。在这种情况下，对应遮蔽 token 的最终隐藏向量会输入到 softmax 函数中，并如标准 LM 中那样预测所有词汇的概率。在所做的所有实验中，我们随机遮住了每个序列中 15% 的 WordPiece token。与去噪自编码器 (Vincent et al., 2008) 相反，我们仅预测遮蔽单词而非重建整个输入。

任务 #2：下一句预测

很多重要的下游任务（如问答（QA）和自然语言推断（NLI））基于对两个文本句子之间关系的理解，这种关系并非通过语言建模直接获得。为了训练一个理解句子关系的模型，我们预训练了一个二值化下一句预测任务，该任务可以从任意单语语料库中轻松生成。具体来说，选择句子 A 和 B 作为预训练样本：B 有 50% 的可能是 A 的下一句，也有 50% 的可能是来自语料库的随机句子。

#### 实验

这部分，我们将展示 BERT 在 11 个 NLP 任务上的微调结果。 

![1539486922211](F:\远传\撰写论文\BERT2.png)